In [1]:
from dataclasses import dataclass

In [2]:
class RSA_ciphertext(int):
    def decrypt(self, privkey):
        return privkey.decrypt(self)

In [3]:
@dataclass
class RSA_privkey:
    n: int
    d: int

    def __str__(self):
        f = "<RSA private key, n={}, d={}>"
        return f.format(self.n, self.d)

    def decrypt(self, C):
        # M = C^d mod n
        M = power_mod(C, self.d, self.n)
        return M

In [4]:
@dataclass
class RSA_pubkey:
    n: int
    e: int
    
    def __str__(self):
        f = "<RSA public key, n={}, e={}>"
        return f.format(self.n, self.e)

    def encrypt(self, C):
        # M = C^d mod n
        M = power_mod(C, self.e, self.n)
        return RSA_ciphertext(M)

In [5]:
class RSA_keypair:
    def __init__(self, n, e, d):
        self.privkey = RSA_privkey(n, d)
        self.pubkey  = RSA_pubkey(n, e)

    def __str__(self):
        f = "<RSA keypair, n={}, e={}, d={}>"
        return f.format(self.pubkey.n, self.pubkey.e, self.privkey.d)

    def encrypt(self, M):
        return self.pubkey.encrypt(M)

    def decrypt(self, C):
        return self.privkey.decrypt(C)

    @staticmethod
    def get_prime(n):
        return random_prime(2^n-1, lbound = 2^(n-1))
    
    @staticmethod
    def generate_primes(bit_len):
        assert bit_len.is_integer()
        bit_len_p = bit_len // 2
        while True:
            p, q = RSA_keypair.get_prime(bit_len_p), RSA_keypair.get_prime(bit_len_p)
            n = (p * q)
            if n.nbits() == bit_len:
                return n, p, q

    @staticmethod
    def compute_phi(p, q):
        return (p - 1) * (q - 1)

    @staticmethod
    def generate_pub_exponent(phi):
        while True:
            # Generate random number within the range
            i = ZZ.random_element(1, phi)
            # Check whether they are coprime
            if gcd(i, phi) == 1: return i

    @staticmethod
    def generate_keypair(bit_len):
        n, p, q = RSA_keypair.generate_primes(bit_len)
        phi = RSA_keypair.compute_phi(p, q)
        e = RSA_keypair.generate_pub_exponent(phi)
        d = e.inverse_mod(phi)

        return RSA_keypair(n, e, d)

In [6]:
def RSA_decrypt(pubkey: RSA_pubkey, C): # Also for verifying
    return pubkey.decrypt(C)

In [7]:
def RSA_encrypt(privkey: RSA_privkey, M): # Also for signing
    return privkey.encrypt(M)

In [8]:
keypair = RSA_keypair.generate_keypair(1024)
print(keypair)
print()
print(keypair.privkey)
print()
print(keypair.pubkey)

<RSA keypair, n=108255997753156503407398709438808355652001845352835348913142378353596120710059679929631283473523867844423355027574494762397260378118604331169817448415395688508971614806003107323741878392551569693817929099076817543620565396191131000247702087609360983553142009795104385579647875345907421942933517047150298635759, e=104765905838621913768644074172257723032817825864655321710164727064265966246066029396142261732378257122516219291877341858242491593832110491654525845413757694069886194868535447029112647738211906071032102171527471153769179399707447119811882630465301785641168847970304473640880110257362480611429628937323011017107, d=37208087529539973405521467425164195269915661166934637799025761085978800668783161951895398154655879706622138474860365597635282479876180242293331167778773590793811794188092704026263193153732778456492104314845552898734183264010969886704145713605948549753474926248577338769880277793140371687425415517828052672243>

<RSA private key, n=108255997753156503407398709

In [9]:
M = 123456789123456789
C = RSA_encrypt(keypair,M); print(C)
M_c = C.decrypt(keypair); print(M_c)

28015538622997534486200553940523917578883875452497896878944198878714458069829335564992314092649418839442295914183912054524098949729264881424458022682486903036755239987160029126591552699431439887486779119768999704193100843530390974935782128085464375337598566402231755014593186436651122726288484244614835325883
123456789123456789


In [10]:
S = RSA_decrypt(keypair, M); print(S)
M_s = RSA_encrypt(keypair, S); print(M_s)

15933564513332037670959000392763657980395206538699410155331599836308190306825612098156113058549617202899108241891854356147069618546488958366941485585008523319047193000038158290360442873093300083009743244064244262005834368917549641228358412507271624645935804478205485445200567481737735454424256974787544463304
123456789123456789


In [11]:
def string2dec(s):                                 # convert message to dec equivalent unicode
    s = [ord(x) for x in s]
    dec = ZZ(list(reversed(s)), 100)
    return dec

In [12]:
def dec2string(dec):                               # convert dec unicodes to string
    s=""
    c=str(dec)
    for i in range(0, len(c),2):
        s+=chr(int(c[i]+c[i+1]))
    return s

In [13]:
M = string2dec("THISCOURSEISSWEET"); print(M)
e=64255123840400920899027594472669379991947555233276781048024250446814302724731106766756339951840237250101891004978212772044383279733379388530493970046211716642208132967525408001754737474165048819205940814225563670212428223096158964837385053632266056026180373587399129459500483194485061624816719090524042421763
n=97317137949299047329422751336372219280650918523616703697935791230362792144624081367345766809355055092002612176057390521301555357528750568302355710548072008293590338833400582094378987467323239514677249799942761914148058633598699435756505298231622102909631623964850403359198735122550801625741661325980099678879
pubkey = RSA_pubkey(n, e); print(pubkey)
C = RSA_encrypt(pubkey, M); print(C)

8472738367798582836973838387696984
<RSA public key, n=97317137949299047329422751336372219280650918523616703697935791230362792144624081367345766809355055092002612176057390521301555357528750568302355710548072008293590338833400582094378987467323239514677249799942761914148058633598699435756505298231622102909631623964850403359198735122550801625741661325980099678879, e=64255123840400920899027594472669379991947555233276781048024250446814302724731106766756339951840237250101891004978212772044383279733379388530493970046211716642208132967525408001754737474165048819205940814225563670212428223096158964837385053632266056026180373587399129459500483194485061624816719090524042421763>
45802885872367657549213178380006301502138720370213810531132025668005540883787882774946139136181409067259858049117360643127831821207406493759858753338563689190380814363291261529979642104897059794938648015368095497654619656703595179296045321636341472406497303938729706422733188523020840238932171954010584687880


In [14]:
d=38628517638999333063216959102132463426514345770457464811914738192908607073140435869189633977768965549270823189579079802564639670293889500507465958035425848348439059951225925511151287285978292921440075521027353133263853532819549925542203782016026084991222393450728182950755020367214717128389736568387727954523
privkey = RSA_privkey(n, d)
M_c = RSA_decrypt(privkey, C); print(M_c)
print(dec2string(M_c))

8472738367798582836973838387696984
THISCOURSEISSWEET


In [15]:
keypair = RSA_keypair(209, 17, 53); print(keypair)
privkey = keypair.privkey
pubkey  = keypair.pubkey

<RSA keypair, n=209, e=17, d=53>


In [16]:
M = 30
C = RSA_encrypt(pubkey, M); print(C)

178


In [17]:
C_mul2 = mod(C*(2^pubkey.e), pubkey.n); print(C_mul2)
C_mul3 = mod(C*(3^pubkey.e), pubkey.n); print(C_mul3)

146
205


In [18]:
M_c = RSA_decrypt(privkey, C); print(M_c)
M_mul2 = RSA_decrypt(privkey, C_mul2); print(M_mul2)
M_mul3 = RSA_decrypt(privkey, C_mul3); print(M_mul3)

30
60
90
